In [1]:
import os
import opensmile
import numpy as np
import sklearn.svm as svm

In [2]:
def extract_audio_feature(dir):
    features = []
    smile = opensmile.Smile(
        feature_set = opensmile.FeatureSet.ComParE_2016,
        feature_level = opensmile.FeatureLevel.Functionals
    )
    folders = os.listdir(dir)
    for folder in folders:
        for i in range(1, 4):
            audio_file = os.path.join(dir, folder, "chapter"+str(i)+".wav")
            temp = smile.process_file(audio_file)
            temp = np.array(temp)
            features.append(temp)
    return features

In [3]:
labels = ['normal', 'mild', 'severe', 'high']

src_dir = 'train_data'
normal_feat = []
mild_feat = []
severe_feat = []
high_feat = []

normal_feat = extract_audio_feature(os.path.join(src_dir, labels[0]))
mild_feat = extract_audio_feature(os.path.join(src_dir, labels[1]))
severe_feat = extract_audio_feature(os.path.join(src_dir, labels[2]))
high_feat = extract_audio_feature(os.path.join(src_dir, labels[3]))

normal_feat = np.array(normal_feat).squeeze()
mild_feat = np.array(mild_feat).squeeze()
severe_feat = np.array(severe_feat).squeeze()
high_feat = np.array(high_feat).squeeze()

In [4]:
X = np.append(normal_feat, mild_feat, axis=0)
X = np.append(X, severe_feat, axis=0)
X = np.append(X, high_feat, axis=0)

Y = np.zeros(shape=(X.shape[0],))
for i in range(len(Y)):
    if i < len(normal_feat):
        Y[i] = 1
    elif i < len(normal_feat) + len(mild_feat):
        Y[i] = 2
    elif i < len(normal_feat) + len(mild_feat) + len(severe_feat):
        Y[i] = 3
    else:
        Y[i] = 4

In [11]:
# linear kernel
C = 1 # SVM regularization parameter
#clf = svm.SVC(kernel="linear", C=C)
clf = svm.SVC(kernel="rbf", C=C, max_iter=10000)
clf = svm.SVC()
clf.fit(X,Y)

SVC()

In [12]:
# confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = clf.predict(X)
confusion_matrix(Y, y_pred)

array([[26,  0, 25,  0],
       [18,  0, 39,  0],
       [21,  0, 45,  0],
       [ 7,  0,  8,  0]], dtype=int64)